# AI Agents

In this section, we'll explore what makes an AI system "agentic" and why building these systems reliably is more challenging than simple chains. You'll:

- Understand LLM agency
- Cover a bit of context engineering
- Create some tools
- Build a couple of durable AI agents (non-looping, and looping)

### Instructor-Led Demo (Expand for instructor notes or to run on your own)
<!--
1. Clone this repository: `https://github.com/cdavisafc/non-deterministic-ai-agents`. The instructions will also be in the README.
2. Run the Worker: `uv run python -m worker`
3. Start the agent: `uv run python -m start_workflow`
4. Emphasize that this demo showcases the core principle of agentic applications: we give **agency** to the **LLM** to drive the flow of the application. Unlike traditional applications where developers hardcode every decision and branching path, this agent makes its own choices at runtime. The LLM decides which actions to take, when to gather more information, and when the goal has been achieved—all autonomously.
-->

### Let's Build An Agent!

First, let's set up your notebook. Run the following code blocks to install various packages and tools necessary to run this notebook

### Notebook Setup

Run the following code blocks to install various packages and tools necessary to run this notebook

**Notice that we must use the OpenAI API key for this chapter**.

In [ ]:
# Create .env file
with open("../.env", "w") as fh:
  fh.write("OPENAI_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

# Now open the file and replace YOUR_API_KEY with your API key.

### Add Your OpenAI API Key **Before** Running the Following Code Block

In [ ]:
# Load environment variables and configure LLM settings
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get OPENAI_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
print("OPENAI_API_KEY", OPENAI_API_KEY)

### Make Sure Your Temporal Web UI is Running

1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

### Building Our Agent: Step by Step

#### Step 1 - Create the Request Dataclass

First, we'll create a dataclass that packages all the parameters needed for our LLM calls. This groups together the model name, agent instructions, conversation input, and available tools in one structure. 

In [ ]:
# TODO: Run this code block to load it into the program
from dataclasses import dataclass
from typing import Any

@dataclass
class OpenAIResponsesRequest:
    model: str
    instructions: str
    input: object
    tools: list[dict[str, Any]]

### Step 2 - Create the Activity for LLM Invocations

We will create an Activity which invokes the OpenAI LLM, taking in our `OpenAIResponsesRequest` dataclass.

It's a reusable function that handles all communication with OpenAI's LLM throughout our agent's execution.


In [ ]:
# Step 1: Supply the `OpenAIResponsesRequest` we just loaded in as the `request` for the parameter
# Step 2: Run this code block to load it into the program
from temporalio import activity
from openai import AsyncOpenAI
from openai.types.responses import Response

@activity.defn
async def create(request: ) -> Response: # TODO: Supply the `OpenAIResponsesRequest` as the `request` for the parameter
    # Temporal best practice: Disable retry logic in OpenAI API client library.
    client = AsyncOpenAI(max_retries=0)

    resp = await client.responses.create(
        model=request.model,
        instructions=request.instructions,
        input=request.input,
        tools=request.tools,
        timeout=30,
    )

    return resp

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "create_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 3 - Create the Helper Function

Before we build our tool that fetches weather alerts, we need a helper function that converts Pydantic models into OpenAI's tool definition format.

The `oai_responses_tool_from_model` function accepts a tool name, description, and a Pydantic model, then returns JSON in the format expected by OpenAI's Responses API tool definitions.

In [ ]:
# TODO: Run this code block
from openai.lib._pydantic import to_strict_json_schema
from pydantic import BaseModel

def oai_responses_tool_from_model(name: str, description: str, model: type[BaseModel]):
    return {
        "type": "function",
        "name": name,
        "description": description,
        # OpenAI Responses strict tools require a JSON Schema object where
        # additionalProperties is explicitly false. For tools without
        # parameters, supply an empty object schema.
        "parameters": (
            to_strict_json_schema(model)
            if model
            else {"type": "object", "properties": {}, "required": [], "additionalProperties": False}
        ),
        "strict": True,
    }

### Step 4 - Set Up the National Weather Service API Wrapper

Before our agent can fetch weather alerts, we need to create helper functions that know how to communicate with the [National Weather Service API](https://www.weather.gov/documentation/services-web-api), specifically for the weather alerts endpoint for a certain state.

These helper functions will be used by our Activity in the next steps.

In [ ]:
# TODO: Run this code block
from typing import Any
import httpx
import json

# Constants for the National Weather Service API
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

def _alerts_url(state: str) -> str:
    """Build the NWS API URL for a given state."""
    return f"{NWS_API_BASE}/alerts/active/area/{state}"

# External calls will happen via activities
async def _make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        response = await client.get(url, headers=headers, timeout=5.0)
        response.raise_for_status()
        return response.json()

### Step 5 - Build the Tool for the OpenAI Responses API

Now we'll create the Pydantic model that defines what parameters the tool accepts, then use our helper function to convert it into OpenAI's tool definition format. 

We will assign this to our tool registry, `WEATHER_ALERTS_TOOL_OAI`, for later use.

In [ ]:
# Step 1: In the `description` for state, set it to "Two-letter US state code (e.g. CA, NY)"
# Step 2: Run this code block
from pydantic import BaseModel, Field

class GetWeatherAlertsRequest(BaseModel):
    state: str = Field(description="")  # TODO: Set the description to be "Two-letter US state code (e.g. CA, NY)"

WEATHER_ALERTS_TOOL_OAI: dict[str, Any] = oai_responses_tool_from_model(
    "get_weather_alerts", # name 
    "Get weather alerts for a US state.", # description
    GetWeatherAlertsRequest) # model

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "GetWeatherAlertsRequest_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 6 - Implement the Activity

We'll create the Activity function that executes when the LLM decides to use this tool. This function calls the NWS API and returns the weather alert data.

In [ ]:
# Step 1: After the `await`, call the `_make_nws_request` function you ran earlier.
# Remember: External calls will happen via Activities.
# Step 2: Run this code block
from temporalio import activity

@activity.defn
async def get_weather_alerts(weather_alerts_request: GetWeatherAlertsRequest) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    data = await ""(_alerts_url(weather_alerts_request.state)) # TODO: Call the `_make_nws_request` function you ran earlier
    return json.dumps(data)

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "get_weather_alerts_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 7: Create the Agent

Now we'll create the agent, which is implemented as the **ToolCallingWorkflow**. This Workflow ties everything together:

1. **Makes an initial LLM call** with the user's input and available tools
2. **Checks if the LLM wants to call a tool** by examining the response type
3. **Executes the tool** if requested
4. **Makes a second LLM call** with the tool results to get a final response

In [ ]:
# Step 1: Call the `create` Activity for the initial LLM call
# Step 2: Fill in the `OpenAIResponsesRequest` with `system_instructions`, `input_list`, and `WEATHER_ALERTS_TOOL_OAI`.
# Step 3: Execute the get_weather_alerts activity when a tool is called
# Step 4: Run this code block to load it into the program

from temporalio import workflow
from datetime import timedelta
import json

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class ToolCallingWorkflow:
    @workflow.run
    async def run(self, input: str) -> str:
        input_list = [{"role": "user", "content": input}]
        
        # Initial LLM call with system instructions and tools
        system_instructions = "if no tools seem to be needed, respond in haikus."
        result = await workflow.execute_activity(
            # TODO: Call the `create` Activity here
            OpenAIResponsesRequest(
                model="gpt-4o-mini",
                instructions="",  # TODO: Fill in with system_instructions
                input="",  # TODO: Fill in with input_list
                tools=[]  # TODO: Fill in tools list with WEATHER_ALERTS_TOOL_OAI
            ),
            start_to_close_timeout=timedelta(seconds=30),
        )
        
        # Process the LLM response
        item = result.output[0]
        
        # if the result is a tool call, call the tool
        if item.type == "function_call":
            if item.name == "get_weather_alerts":

                # serialize the output, which is an OpenAI object
                input_list += [
                    i.model_dump() if hasattr(i, "model_dump") else i
                    for i in result.output
                ]
                
                result = await workflow.execute_activity(
                    "", # TODO: Execute the get_weather_alerts activity
                    GetWeatherAlertsRequest(state=json.loads(item.arguments)["state"]),
                    start_to_close_timeout=timedelta(seconds=30),
                )
                
                # Add tool call result to input list
                input_list.append({
                    "type": "function_call_output",
                    "call_id": item.call_id,
                    "output": result
                })
                
                result = await workflow.execute_activity(
                    create,
                    OpenAIResponsesRequest(
                        model="gpt-4o-mini",
                        instructions="return the tool call result in a readable format",
                        input=input_list,
                        tools=[]
                    ),
                    start_to_close_timeout=timedelta(seconds=30),
                )
        
        result = result.output_text
        return result

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "tool_calling_workflow_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Running the Tool-Calling Agent

Now let's run our agent by running our Worker!

In [ ]:
# Step 1: Set the Task Queue to be "tool-calling-python-task-queue"
# Step 2: Register `ToolCallingWorkflow` on this Worker
# Step 3: Run this code
import concurrent.futures
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.contrib.pydantic import pydantic_data_converter

async def run_worker() -> None:
    # Create client connected to server at the given address
    client = await Client.connect("localhost:7233", data_converter=pydantic_data_converter)

    # Run the Worker
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as activity_executor:
        worker = Worker(
            client,
            task_queue="", # TODO Set the Task Queue to be "tool-calling-python-task-queue"
            workflows=[], # TODO Register `ToolCallingWorkflow` on this Worker
            activities=[create, get_weather_alerts],
            activity_executor=activity_executor
        )

        print(f"Starting the worker....")
        await worker.run()

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "worker_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Kill the old worker and start the new one
import asyncio

worker = asyncio.create_task(run_worker())

### Initiate an Interaction with the Agent

In order to interact with this simple AI agent, we create a Temporal client and execute a workflow.

In [ ]:
# Step 1: Set the task queue of the client to match the task queue the worker is polling
# Step 2: Run this code block
import sys
import uuid

from temporalio.client import Client
from temporalio.contrib.pydantic import pydantic_data_converter

async def main():
    client = await Client.connect(
        "localhost:7233",
        data_converter=pydantic_data_converter,
    )

    query = sys.argv[1] if len(sys.argv) > 1 else "Hello, how are you?"

    # Submit the Tool Calling workflow for execution
    result = await client.execute_workflow(
        ToolCallingWorkflow,
        query,
        id=f"weather-alert-agent-{uuid.uuid4()}",
        task_queue="", # TODO: Set the task queue of the client to match the task queue the worker is polling
    )
    print(f"Result: {result}")

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "client_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Test the Agent

Now let's test the agent with different queries. Run the cells below to see how the agent responds to different types of requests.

In [ ]:
# Test with California weather
import sys

sys.argv = ["", "Are there any weather alerts in CA?"]
await main()

In [ ]:
# Test with New York weather
import sys

sys.argv = ["", "Are there any weather alerts in NY?"]
await main()

In [ ]:
# Test without tools (should get haiku)
import sys

sys.argv = ["", "Tell me a joke"]
await main()

## Observe Your Web UI

Refresh your Web UI. 

- What Activities are called for a weather alert Workflow Execution vs. the joke Workflow Execution?
- Click on the first create Activity in the weather alert Workflow. What was passed in the input field?
- Click on the second create Activity in the weather alert Workflow.  How has the input field changed?

A point to make is that the **LLM decides autonomously** whether to use the weather
alerts tool or not. You don't hardcode "if user mentions a state, call the API" -
instead, the LLM evaluates the input and decides if calling `get_weather_alerts`
will help achieve the goal.

In [ ]:
# Kill any Workers to prepare for the next part:

worker.cancel() 

## Building a Looping Agent with Dynamic Tools

In the previous section, you built a simple tool-calling agent that could handle one tool call and return a response. Now, let's build the **looping agent** that demonstrates:

1. **An agentic loop** - continuously consults the LLM until the task is done
2. **Dynamic tool invocation** - the workflow doesn't need separate code for each tool. Instead, one dynamic activity automatically routes to the correct tool function based on which tool the LLM chooses   
3. **Multiple tool support** - weather alerts, random numbers, IP location lookup

### Step 1: Add More Tools

First, let's add additional tools from the simple-agent repository. We'll add:
- **Random number generator** - generates random numbers
- **IP address lookup** - gets the current machine's IP
- **Location lookup** - gets location info from an IP address

In [ ]:
# TODO: Run this code block to load the tool implementation functions
from pydantic import BaseModel, Field
import requests, random

# IP and Location tools
class GetLocationRequest(BaseModel):
    ipaddress: str = Field(description="An IP address")

async def get_random_number() -> str:
    """Get a random number between 0 and 100."""
    data = random.randint(0, 100)
    return str(data)

def get_ip_address() -> str:
    """Get the IP address of the current machine."""
    response = requests.get("https://icanhazip.com")
    response.raise_for_status()
    return response.text.strip()

def get_location_info(req: GetLocationRequest) -> str:
    """Get location information for an IP address."""
    response = requests.get(f"http://ip-api.com/json/{req.ipaddress}")
    response.raise_for_status()
    result = response.json()
    return f"{result['city']}, {result['regionName']}, {result['country']}"

In [ ]:
# Step 1: Fill in the description for get_random_number tool
# Step 2: Fill in the description for get_ip_address tool
# Step 3: Fill in the description for get_location_info tool
# Step 4: Run this code block

# Random number tool
RANDOM_NUMBER_TOOL_OAI: dict[str, Any] = oai_responses_tool_from_model(
    "get_random_number",
    "",  # TODO: Add description similar to "Get a random number between 0 and 100."
    None)  # No parameters needed

GET_IP_ADDRESS_TOOL_OAI: dict[str, Any] = oai_responses_tool_from_model(
    "get_ip_address",
    "",  # TODO: Add description similar to "Get the IP address of the current machine."
    None)

GET_LOCATION_TOOL_OAI: dict[str, Any] = oai_responses_tool_from_model(
    "get_location_info",
    "",  # TODO: Add description similar to "Get the location information for an IP address. This includes the city, state, and country."
    GetLocationRequest)

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "tools_solution.py"

code = solution_file.read_text()
print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Step 1: Add @activity.defn(dynamic=True) decorator  
# Step 2: Get tool_name from activity.info().activity_type
# Step 3: Call get_handler() to get the handler function
# Step 4: Run this code block

from temporalio import activity
from typing import Sequence
from temporalio.common import RawValue
import inspect
from pydantic import BaseModel

# Tool handler registry - maps tool names to functions
def get_handler(tool_name: str):
    if tool_name == "get_location_info":
        return get_location_info
    if tool_name == "get_ip_address":
        return get_ip_address
    if tool_name == "get_weather_alerts":
        return get_weather_alerts
    if tool_name == "get_random_number":
        return get_random_number

# TODO: Add @activity.defn(dynamic=True) decorator
async def dynamic_tool_activity(args: Sequence[RawValue]) -> dict:
    
    # TODO: Get the tool name - use activity.info().activity_type 
    tool_name = ""
    
    # Convert the raw Temporal payload to a Python dictionary
    tool_args = activity.payload_converter().from_payload(args[0].payload, dict)
    activity.logger.info(f"Running dynamic tool '{tool_name}' with args: {tool_args}")

    # TODO: Get the handler using get_handler(tool_name)
    handler = ""
    
    # Inspect the handler's function signature to see what parameters it expects
    sig = inspect.signature(handler)
    params = list(sig.parameters.values())

    # If the handler takes no parameters, pass an empty list
    if len(params) == 0:
        call_args = []
    else:
        # If the handler expects a Pydantic model, convert the dict to that model
        ann = params[0].annotation
        if isinstance(tool_args, dict) and isinstance(ann, type) and issubclass(ann, BaseModel):
            call_args = [ann(**tool_args)]
        else:
            call_args = [tool_args]

    # Call the handler (handle both async and sync functions)
    result = await handler(*call_args) if inspect.iscoroutinefunction(handler) else handler(*call_args)

    activity.logger.info(f"Tool '{tool_name}' result: {result}")
    return result

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "dynamic_activity_solution.py"

code = solution_file.read_text()
print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 3: Create the System Instructions

The simple-agent uses specific system instructions. Let's add those now:

In [ ]:
# Step 1: In the `get_tools` method, add `RANDOM_NUMBER_TOOL_OAI`, `GET_LOCATION_TOOL_OAI`, `GET_IP_ADDRESS_TOOL_OA` to the tool list
# Step 2: Run this code block

HELPFUL_AGENT_SYSTEM_INSTRUCTIONS = """
You are a helpful agent that can use tools to help the user.
You will be given a task and a list of tools to use.
You may or may not need to use the tools to complete the task.
If no tools are needed, respond in haikus.
"""

# Create the tool list for the agent
def get_tools():
    return [WEATHER_ALERTS_TOOL_OAI, 
            # TODO: add `RANDOM_NUMBER_TOOL_OAI`, `GET_LOCATION_TOOL_OAI`, `GET_IP_ADDRESS_TOOL_OA` to the tool list
            ]

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "system_instructions_solution.py"

code = solution_file.read_text()
print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 4: Build the Agent Workflow

Now let's create the Agent Workflow. This is the **agentic loop** - it keeps calling the LLM and executing tools until the LLM decides the task is complete.

In [ ]:
# Step 1: Create a while True loop for the agentic loop
# Step 2: Check if item.type == "function_call" to detect tool calls
# Step 3: Call _handle_function_call when a tool is requested
# Step 4: Run this code block

from temporalio import workflow
from datetime import timedelta
import json

@workflow.defn(sandboxed=False)
class AgentWorkflow:
    @workflow.run
    async def run(self, input: str) -> str:

        input_list = [{"type": "message", "role": "user", "content": input}]

        # TODO: Create a while True loop
        _____ True:

            print(80 * "=")
                
            # consult the LLM
            result = await workflow.execute_activity(
                create,
                OpenAIResponsesRequest(
                    model="gpt-4o-mini",
                    instructions=HELPFUL_AGENT_SYSTEM_INSTRUCTIONS,
                    input=input_list,
                    tools=get_tools(),
                ),
                start_to_close_timeout=timedelta(seconds=30),
            )

            item = result.output[0]
            
            # TODO: Check if the item type is "function_call"
            if item.type == "":
                result = await self._handle_function_call(item, result, input_list)
                
                # add the tool call result to the input list for context
                input_list.append({"type": "function_call_output",
                                    "call_id": item.call_id,
                                    "output": result})

            else:
                print(f"No tools chosen, responding with a message: {result.output_text}")
                return result.output_text

    async def _handle_function_call(self, item, result, input_list):
        # serialize the LLM output - the decision the LLM made to call a tool
        i = result.output[0]
        input_list += [
            i.model_dump() if hasattr(i, "model_dump") else i
        ]
        # execute dynamic activity with the tool name chosen by the LLM
        args = json.loads(item.arguments) if isinstance(item.arguments, str) else item.arguments

        # TODO: Call workflow.execute_activity with item.name as the activity
        result = await workflow.execute_activity(
            ,  # TODO: Use item.name to dynamically call the tool
            args,
            start_to_close_timeout=timedelta(seconds=30),
        )

        print(f"Made a tool call to {item.name}")
        return result

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "agent_workflow_solution.py"

code = solution_file.read_text()
print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 5: Create the Worker

Now let's create the Worker to run our agent.

In [ ]:
# Step 1: Set task_queue to "chaotic-agent-python-task-queue"
# Step 2: Add AgentWorkflow to the workflows list
# Step 3: Add both create and dynamic_tool_activity to activities
# Step 4: Run this code block

from concurrent.futures import ThreadPoolExecutor
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.contrib.pydantic import pydantic_data_converter

async def run_agent_worker():
    client = await Client.connect(
        "localhost:7233",
        data_converter=pydantic_data_converter,
    )

    worker = Worker(
        client,
        task_queue="", # TODO: Set to "chaotic-agent-python-task-queue"
        workflows=[], # TODO: Add AgentWorkflow
        activities=[], # TODO: Add create and dynamic_tool_activity
        activity_executor=ThreadPoolExecutor(max_workers=10),
    )
    
    print("Starting agent worker...")
    await worker.run()

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "agent_worker_solution.py"

code = solution_file.read_text()
print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Cancel the previous worker and start the agent worker
import asyncio

worker.cancel()

agent_worker = asyncio.create_task(run_agent_worker())

### Step 6: Create the Client to Start Workflows

Finally, let's create the client.

In [ ]:
# Step 1: Execute AgentWorkflow.run
# Step 2: Set task_queue to match the worker
# Step 3: Run this code block

import uuid
from temporalio.client import Client
from temporalio.contrib.pydantic import pydantic_data_converter

async def start_agent(query: str):
    client = await Client.connect(
        "localhost:7233",
        data_converter=pydantic_data_converter,
    )

    # TODO: Execute AgentWorkflow
    result = await client.execute_workflow(
        ,  # TODO: Add AgentWorkflow
        query,
        id=f"my-workflow-id-{uuid.uuid4()}",
        task_queue="",  # TODO: Set to "chaotic-agent-python-task-queue"
    )
    print(f"Result: {result}")
    return result

In [ ]:
# Optional: Run this cell to display the solution, which you can copy and paste over the code block above.
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "start_workflow_solution.py"

code = solution_file.read_text()
print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 7: Test the Agent!

Now let's test our agent with different queries. Note that running the agent can take ~1 minute for the Workflow to complete and to get a result. Notice how it can use multiple tools!

In [ ]:
# Test with weather alerts
await start_agent("What are the weather alerts in California?")

### What Makes This a True Agentic Loop?

Observe how the agent:
1. **Continuously loops** - calls the LLM → executes tools → calls LLM again until done
2. **Makes autonomous decisions** - the LLM decides which tools to use and when
3. **Handles multiple tools** - can chain tool calls together (e.g., get IP → get location)
4. **Exits gracefully** - returns a response when no more tools are needed

In [ ]:
# Test without tools (should get haiku)
await start_agent("Tell me about penguins")

In [ ]:
# Test with location lookup (will use multiple tools!)
await start_agent("What's my location?")

In [ ]:
# Test with random number
await start_agent("Give me a random number")

### Bonus Challenge: Add More Tools!

Want to make your agent even more capable? Try adding additional tools. You can work from this [simple-agent
repository](https://github.com/cdavisafc/simple-agent/)!

**Ideas for new tools:**
1. **Get Current Time** - Return the current date and time in a specific timezone  
2.  **Calculate Math** - Evaluate mathematical expressions (e.g., "2 + 2", "sqrt(16)")   
3. **Your own tool!** - What would be useful for your use case?

Try it out and see how the agent's capabilities expand!

## Thank you for attending this workshop. Feedback?

Please leave your feedback for this workshop [here](https://docs.google.com/forms/d/e/1FAIpQLSfkHMev6KCNGFHpVNydyjgAh2ALeHNVYv9TaSrAoBsT0KmNHQ/viewform?usp=header)!

### What's Next?

Further your learning with these resources:

### Resources

- [Temporal Docs](https://docs.temporal.io/)
- [Free Temporal Courses](https://learn.temporal.io/courses/)
- Learn more about [AI Agents with Temporal](https://temporal.io/code-exchange/ai-agent-execution-using-temporal)
- The [repo for the Agent](https://github.com/cdavisafc/simple-agent/) used in this workshop
- Join our [Community Slack](https://t.mp/slack)